In [27]:
from gurobipy import*

In [28]:
import json
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil


def process_file(file_path):
    
    with open(file_path, 'r') as file:
        data = json.load(file)
        load_actions = data['route']

    # Extract vehicle keys in sorted order
    sorted_vehicle_keys = sorted(data['vehicles'].keys(), key=lambda x: int(x[1:]))

    # Extract the dimensions in sorted order
    vehicle_capacity = np.array([data['vehicles'][vehicle]['dimension'] for vehicle in sorted_vehicle_keys])

    deck_capacity = np.array([data['transporter']['decks'][deck]['capacity'] for deck in sorted(data['transporter']['decks'].keys())])


    # Get the number of stops
    num_stops = len(data['route'])

    # Extract labels from 'decks' keys
    labels = ['Deck ' + str(i + 1) for i in range(len(data['transporter']['decks']))]

    # Number of vehicles
    num_vehicles = len(data['vehicles'])



    # Generate the vehicle_indices dictionary
    vehicle_indices = {vehicle: idx for idx, vehicle in enumerate(data['vehicles'].keys())}


    # Find the maximum deck capacity
    max_deck_capacity = max(deck_capacity)


    # Extract the total capacity
    total_capacity = data['transporter']['total_capacity']


    # Create the deck_via dictionary
    deck_via = {}
    for deck_key, deck_info in data['transporter']['decks'].items():
        # Convert deck key from 'd1' to 1
        deck_number = int(deck_key[1:])
        # Extract accessible decks and convert them from [['d2', 'd4'], ['d3', 'd4']] to [[2, 4], [3, 4]]
        accessible_decks = [[int(d[1:]) for d in sublist] for sublist in deck_info.get('access_via', [])]
        # Assign to deck_via dictionary
        deck_via[deck_number] = accessible_decks

    # Convert dictionary values to NumPy arrays
    deck_via_np = {k: np.array(v) for k, v in deck_via.items()}

    # Initialize the vehicle stops list
    vehicle_stops = []
    
    # Initialize the set of currently loaded vehicles
    current_vehicles = set()

    # Iterate over the load actions to fill vehicle_stops
    for action in load_actions:
        # Unload vehicles if there are any
        if 'unload' in action:
            for vehicle in action['unload']:
                current_vehicles.discard(vehicle_indices[vehicle])
        # Load vehicles if there are any
        if 'load' in action:
            for vehicle in action['load']:
                current_vehicles.add(vehicle_indices[vehicle])

        # Append a sorted list of currently loaded vehicle indices to vehicle_stops
        vehicle_stops.append(sorted(current_vehicles))

    model = Model("feasibility checking")

    x = {}
    for i in range(len(vehicle_capacity)):
        for j in range(len(deck_capacity)):
            x[i,j] = model.addVar(vtype = 'B')

    for k in range(len(vehicle_stops)):
        for j in range(len(deck_capacity)):
            model.addConstr(quicksum(vehicle_capacity[i]*x[i,j]for i in vehicle_stops[k]) <= deck_capacity[j])

    for i in range(len(vehicle_capacity)):
        model.addConstr(quicksum(x[i,j] for j in range(len(deck_capacity))) == 1)


    model.setObjective(quicksum(x[i,j] for i in range(len(vehicle_stops[k])) for j in range(len(deck_capacity))), GRB.MAXIMIZE)
    
    model.setParam('OutputFlag', 0)  # Suppress solver output
    model.optimize()

    # Check the optimization status
    if model.Status == GRB.INFEASIBLE:
        return [file_path, 'NA']
    else:
        filename = os.path.basename(file_path)
        #shutil.move(os.path.join(source_dir, filename), os.path.join(target_dir_s1, filename))
        # If model is feasible, return the objective value or any other result you need
        return [file_path, model.objVal]


In [29]:
# Define the source directory
#source_dir = r'C:\Users\USER\桌面\soft_infeasible'
# Define the target directories
#target_dir_s1 = os.path.join(source_dir, 'soft')

input_files = glob.glob(r'C:\Users\USER\桌面\soft_infeasible\*.json')

inputFile = []
for file_path in input_files:
    if 'solution' not in file_path:
        inputFile.append(file_path)


results = []
for file_path in inputFile:
    result = process_file(file_path)
    if result[1] != 'NA':
        filename = os.path.basename(file_path)
        results.append(filename)
    
    
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
#output_excel_file = r'C:\Users\USER\OneDrive - 國立成功大學 National Cheng Kung University\桌面\soft_infeasible\soft.xlsx'
#df.to_excel(output_excel_file, index=False)

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000
25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400
27600
27800
28000
28200
28400
28600
28800
29000
29200
29400
29600
29800
30000
30200
30400
30600
30800
31000
31200
31400
31600
31800
32000
32200
32400
32600
32800
33000
33200
33400
33600
33800
34000
34200
34400
34600
34800
35000
352